In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.18.1


In [10]:
def load_file(filename):
    fp = open(filename, encoding='utf8')#,'Ur')
    data_list = []
    sent=[]
    x=0
    for line in fp:
        if (line.strip()=='.\t.\tO'):
            sent.append(tuple(line.strip().split('\t')))
            data_list.append(sent)
            sent=[]
        else:
            if(line.strip()==''):
                x=1
            else:
                sent.append(tuple(line.strip().split('\t')))
    fp.close()
    return data_list

        #print (line)
        #data_list.append(tuple(line.strip().split(',')))

In [11]:
#train_sent_smpl=load_file("D:/D drive/DataAnalytics_StudyMaterial/IIMB-BDA/ProjectZ/Iteration2/Train1Sample.csv")
train_sent_smpl=load_file("D:/D drive/DataAnalytics_StudyMaterial/IIMB-BDA/ProjectZ/Iteration2/TrainData.csv")

In [12]:
#test_sent_sampl=load_file("D:/D drive/DataAnalytics_StudyMaterial/IIMB-BDA/ProjectZ/Iteration2/Test1Sample.csv")
test_sent_sampl=load_file("D:/D drive/DataAnalytics_StudyMaterial/IIMB-BDA/ProjectZ/Iteration2/TestData.csv")

In [6]:
#nltk.corpus.conll2000.fileids()

['train.txt', 'test.txt']

In [7]:
%%time
#train_sents = list(nltk.corpus.conll2000.iob_sents('train.txt'))
#test_sents = list(nltk.corpus.conll2000.iob_sents('test.txt'))

Wall time: 4.06 s


In [13]:
train_sent_smpl[1]


[('Hands', 'NNS', 'O'),
 ('on', 'IN', 'O'),
 ('experience', 'NN', 'WRKEX'),
 ('in', 'IN', 'O'),
 ('Java', 'NNP', 'SKILL'),
 ('and', 'CC', 'O'),
 ('J2EE', 'NNP', 'SKILL'),
 ('(JDBC', 'NNP', 'SKILL'),
 ('Servlet', 'NNP', 'SKILL'),
 ('JSP).', 'NNP', 'SKILL'),
 ('.', '.', 'O')]

In [14]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [15]:
len(train_sent_smpl)

2398

In [16]:
len(test_sent_sampl)

191

In [18]:
%%time
X_train = [sent2features(s) for s in train_sent_smpl]
y_train = [sent2labels(s) for s in train_sent_smpl]

X_test = [sent2features(s) for s in test_sent_sampl]
y_test = [sent2labels(s) for s in test_sent_sampl]

Wall time: 632 ms


In [19]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

Wall time: 940 ms


In [35]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 500,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [36]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [22]:
import os

In [23]:
os.getcwd()

'C:\\Users\\Shashank Shekar'

In [37]:
%%time
#trainer.train('conll2002-esp.crfsuite')
trainer.train('TotalCV2.crfsuite')

Wall time: 1min 2s


In [38]:
trainer.logparser.last_iteration

{'active_features': 3738,
 'error_norm': 73.534805,
 'feature_norm': 71.145594,
 'linesearch_step': 0.5,
 'linesearch_trials': 2,
 'loss': 10079.875607,
 'num': 500,
 'scores': {},
 'time': 0.205}

In [39]:
print (len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

500 {'num': 500, 'scores': {}, 'loss': 10079.875607, 'feature_norm': 71.145594, 'error_norm': 73.534805, 'active_features': 3738, 'linesearch_trials': 2, 'linesearch_step': 0.5, 'time': 0.205}


In [40]:
tagger = pycrfsuite.Tagger()
tagger.open('TotalCV2.crfsuite')

In [41]:
example_sent = test_sent_sampl[2]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Extensive knowledge of CoreJava concepts like OOPS Concepts Multithreading Exception Handling Interfaces Abstract Class and Collection Framework. .

Predicted: O O O O O O SKILL SKILL SKILL SKILL SKILL SKILL SKILL SKILL O SKILL SKILL O
Correct:   O O O SKILL O O SKILL SKILL SKILL SKILL SKILL SKILL SKILL SKILL SKILL SKILL SKILL O


In [42]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [43]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

Wall time: 82.1 ms


In [31]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       DATE       0.96      0.77      0.85        91
      DESIG       0.81      0.68      0.74        25
        EDU       0.95      0.46      0.62        46
      EMAIL       1.00      1.00      1.00         7
        LOC       0.88      0.46      0.60        46
        ORG       0.70      0.61      0.65       119
       PERS       0.91      0.62      0.74        16
      PHONE       0.75      0.60      0.67        10
      SKILL       0.83      0.64      0.73       726
      WRKEX       0.93      0.52      0.67        25

avg / total       0.84      0.63      0.72      1111



In [32]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
EMAIL  -> EMAIL   3.757939
EDU    -> EDU     3.633941
WRKEX  -> WRKEX   3.517239
ORG    -> ORG     3.428494
LOC    -> LOC     3.313029
DESIG  -> DESIG   3.215806
PHONE  -> PHONE   3.133305
PERS   -> PERS    2.719366
EMAIL  -> PHONE   2.564856
ORG    -> LOC     2.270312
DATE   -> DATE    2.222121
SKILL  -> SKILL   1.862740
PHONE  -> EMAIL   1.685653
O      -> O       1.564512
PERS   -> EMAIL   0.785996

Top unlikely transitions:
SKILL  -> LOC     -1.920234
DATE   -> SKILL   -1.947272
ORG    -> SKILL   -1.990016
DESIG  -> SKILL   -2.007271
SKILL  -> ORG     -2.038188
PERS   -> ORG     -2.153862
SKILL  -> DESIG   -2.230103
PHONE  -> SKILL   -2.284765
SKILL  -> PHONE   -2.444673
SKILL  -> DATE    -2.457903
WRKEX  -> SKILL   -2.528212
SKILL  -> EDU     -2.983153
SKILL  -> PERS    -3.030185
EDU    -> SKILL   -3.181291
PERS   -> SKILL   -3.211644


In [33]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
6.386089 O      word.lower=using
6.031084 DATE   -1:word.lower=till
4.836144 DATE   word.lower=may
4.832337 SKILL  word.lower=hive
4.650458 ORG    word.lower=telenet
4.328767 LOC    word.lower=bangalore
4.303009 SKILL  word.lower=jquery
4.301115 SKILL  word.lower=angular
4.290237 WRKEX  +1:word.lower=years
4.170748 SKILL  word.lower=spark
4.103823 SKILL  word.lower=wordpress
4.092707 LOC    word.lower=hyderabad
4.075117 ORG    word.lower=hospital
4.069478 SKILL  word.lower=unix
4.055064 O      EOS
3.991890 LOC    word.lower=pune
3.934869 ORG    -1:word.lower=client:
3.856411 DESIG  -1:word.lower=team
3.850966 WRKEX  +1:word.lower=year
3.816833 SKILL  word[-2:]=JS

Top negative:
-1.406615 SKILL  -1:word.lower=java
-1.407054 LOC    word[-2:]=on
-1.426566 PHONE  BOS
-1.441366 O      word.lower=dell
-1.459049 O      word.lower=certificate
-1.566300 SKILL  word.lower=experience
-1.589936 O      word.lower=manual
-1.642699 O      word[-3:]=pal
-1.749610 O      word.lower=hospit